<a href="https://colab.research.google.com/github/dethhe/nlpcc-ui-2025/blob/main/Class_5_%26_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Simple LLM Chat Application (Vanilla Chat)

In [1]:
!pip install openai

In [ ]:
import os
from openai import OpenAI
from google.colab import userdata # Untuk mengambil rahasia dari Colab

# Mengambil kunci API dari rahasia Colab
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except Exception as e:
    print(f"Error retrieving OPENAI_API_KEY: {e}")
    print("Pastikan Anda telah menyimpan kunci API sebagai 'OPENAI_API_KEY' di bagian 'Secrets' Colab.")
    print("Aplikasi tidak akan berfungsi tanpa kunci API yang valid.")
    exit() # Keluar jika kunci API tidak ditemukan

# Inisialisasi klien OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

print("Selamat datang di Obrolan LLM Sederhana!")
print("Ketik 'exit' atau 'keluar' untuk mengakhiri obrolan.")
print("-" * 30)

# Loop obrolan
while True:
    user_input = input("Anda: ")

    if user_input.lower() in ["exit", "keluar"]:
        print("Obrolan berakhir. Sampai jumpa!")
        break

    try:
        # Membuat respons dari LLM
        # Menggunakan model "gpt-3.5-turbo" karena "gpt-4.1" tidak standar dan mungkin tidak tersedia
        # "gpt-4" adalah model yang lebih umum jika Anda memiliki akses
        response = client.chat.completions.create(
            model="gpt-3.5-turbo", # Anda bisa ganti dengan "gpt-4" jika punya akses
            messages=[
                {"role": "user", "content": user_input}
            ]
        )

        # Menampilkan respons dari LLM
        # Respons dari model chat ada di response.choices[0].message.content
        llm_response = response.choices[0].message.content
        print(f"LLM: {llm_response}")

    except Exception as e:
        print(f"Terjadi kesalahan saat memproses permintaan: {e}")
        print("Pastikan kunci API Anda valid dan Anda memiliki koneksi internet.")

Selamat datang di Obrolan LLM Sederhana!
Ketik 'exit' atau 'keluar' untuk mengakhiri obrolan.
------------------------------
Anda: hi bantu saya
LLM: Tentu, ada hal apa yang bisa saya bantu?
Anda: kapan indonesia merdeka
LLM: Indonesia merdeka pada tanggal 17 Agustus 1945.


# Task 2: LLM-Powered Tabular Data Processing

In [ ]:
#-------------------------------------------------------------------------------
# TASK 2: LLM-Powered Tabular Data Processing (using OpenAI)
#-------------------------------------------------------------------------------

# 1. Install and Import Libraries
# !pip install -q pandas openai

import pandas as pd
import openai # Diganti dari google.generativeai
import getpass
import time
from google.colab import drive # Untuk Google Drive

print("--- LLM-Powered Tabular Data Processing (OpenAI) ---")

# --- MOUNT GOOGLE DRIVE ---
try:
    drive.mount('/content/drive', force_remount=True) # force_remount bisa membantu jika ada masalah mount sebelumnya
    print("\nGoogle Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    # exit() # Pertimbangkan untuk menghentikan skrip jika Drive gagal di-mount

# 2. Configure OpenAI API Key
# --- IMPORTANT: GET YOUR OPENAI API KEY ---
# Anda perlu mendapatkan API key dari OpenAI: https://platform.openai.com/account/api-keys
# Ganti "YOUR_OPENAI_API_KEY" dengan kunci Anda atau jalankan sel ini untuk diminta.
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if not OPENAI_API_KEY:
        print("OpenAI API Key not found in Colab secrets.")
        OPENAI_API_KEY = getpass.getpass('Enter your OpenAI API Key: ')
except ImportError:
    OPENAI_API_KEY = getpass.getpass('Enter your OpenAI API Key: ')

openai.api_key = OPENAI_API_KEY

# 3. Load Data
# --- PERIKSA KEMBALI PATH FILE INI DI GOOGLE DRIVE ANDA ---
file_path_on_drive = '/content/drive/MyDrive/drive/Review Crocs Fiktif.csv' # Path yang Anda berikan
df = None

try:
    print(f"\nAttempting to load file from: {file_path_on_drive}")
    df = pd.read_csv(file_path_on_drive)
    print(f"Successfully loaded '{file_path_on_drive}'. Shape: {df.shape}")
    print("First 3 rows of the dataset:")
    print(df.head(3))
except FileNotFoundError:
    print(f"ERROR: File '{file_path_on_drive}' not found. ")
    print("Please double-check the following:")
    print("1. The Google Drive was mounted successfully (see message above).")
    print("2. The file path is EXACTLY correct (case-sensitive, no typos).")
    print("   Correct path should be like: /content/drive/MyDrive/NAMA_FOLDER_ANDA/Review Crocs Fiktif.csv")
    print("   Jika file Anda ada di root MyDrive, pathnya: /content/drive/MyDrive/Review Crocs Fiktif.csv")
    print("   Jika file Anda ada di folder 'data' di MyDrive: /content/drive/MyDrive/data/Review Crocs Fiktif.csv")
    # exit()
except Exception as e:
    print(f"An error occurred while loading the file: {e}")
    # exit()

# Proceed only if df is loaded successfully
if df is not None:
    # 4. Sample Data
    if len(df) > 15:
        df_sample = df.sample(n=15, random_state=42)
    else:
        df_sample = df.copy()
    print(f"\nProcessing a sample of {len(df_sample)} rows.")

    # 5. LLM Interaction (OpenAI)
    sentiments_llm = []
    summaries_llm = []

    print("\nProcessing rows with OpenAI LLM (this may take a moment)...")
    for index, row in df_sample.iterrows():
        review_text = row['Ulasan']

        # --- Task 1: Sentiment Analysis ---
        prompt_sentiment = f"""
        Analisis sentimen dari ulasan produk berikut.
        Ulasan: "{review_text}"
        Jawab HANYA dengan salah satu dari: 'Positif', 'Negatif', atau 'Netral'.
        Sentimen:
        """

        # --- Task 2: Summarization ---
        prompt_summary = f"""
        Buatkan ringkasan singkat (maksimal 1-2 kalimat) dari ulasan produk berikut:
        Ulasan: "{review_text}"
        Ringkasan:
        """

        current_sentiment = "Error: API call failed"
        current_summary = "Error: API call failed"

        try:
            # Sentiment Analysis with OpenAI
            response_sentiment = openai.ChatCompletion.create(
                model="gpt-3.5-turbo", # Anda bisa ganti dengan model lain jika perlu
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that analyzes sentiment."},
                    {"role": "user", "content": prompt_sentiment}
                ],
                temperature=0.2 # Suhu rendah untuk jawaban yang lebih konsisten
            )
            current_sentiment = response_sentiment.choices[0].message.content.strip()
            # Basic cleaning
            if "Positif" in current_sentiment: current_sentiment = "Positif"
            elif "Negatif" in current_sentiment: current_sentiment = "Negatif"
            elif "Netral" in current_sentiment: current_sentiment = "Netral"
            else: current_sentiment = "Netral (Unclear)"

            # Summarization with OpenAI
            response_summary = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that summarizes text."},
                    {"role": "user", "content": prompt_summary}
                ],
                temperature=0.5
            )
            current_summary = response_summary.choices[0].message.content.strip()

            print(f"Processed row {df_sample.index.get_loc(index)+1}/{len(df_sample)}: Sentiment='{current_sentiment}'")
            time.sleep(1) # Jeda untuk menghindari rate limit API

        except openai.error.RateLimitError:
            print(f"OpenAI API rate limit exceeded for row with index {index}. Waiting and retrying...")
            time.sleep(20) # Tunggu lebih lama jika kena rate limit
            # Anda bisa menambahkan logika retry di sini jika mau
            current_sentiment = "Error: Rate limit"
            current_summary = "Error: Rate limit"
        except Exception as e:
            print(f"Error processing row with index {index} for review: '{str(review_text)[:50]}...': {e}")
            current_sentiment = f"Error: {str(e)}"
            current_summary = f"Error: {str(e)}"


        sentiments_llm.append(current_sentiment)
        summaries_llm.append(current_summary)

    # Add the LLM responses to the sample DataFrame
    df_sample['Sentimen_LLM_OpenAI'] = sentiments_llm
    df_sample['Ringkasan_LLM_OpenAI'] = summaries_llm

    # 7. Display Results
    print("\n--- Results (OpenAI) ---")
    print("Original data with LLM-generated sentiment and summary:")

    pd.set_option('display.max_colwidth', 200)
    display_df = df_sample[['Topik', 'Ulasan', 'Sentimen_LLM_OpenAI', 'Ringkasan_LLM_OpenAI']]
    print(display_df)

    # output_filename_openai = "/content/drive/MyDrive/drive/Review_Crocs_Fiktif_Processed_OpenAI.csv"
    # display_df.to_csv(output_filename_openai, index=False)
    # print(f"\nResults saved to {output_filename_openai}")

else:
    print("\nSkipping LLM processing as DataFrame 'df' was not loaded due to file error.")